In [1]:
%matplotlib inline
import os
import ipywidgets as widgets
from matplotlib import pyplot as plt

file_path = widgets.Text(
    description = 'path'
)

display(file_path)

Text(value='', description='path')

In [6]:
import configparser
config = configparser.ConfigParser()
config.read(file_path.value)

im_size = int(config['Training']['im_size'])
raw_path = config['Training']['raw_path']
hr_path = config['Training']['hr_path']
lr_path = config['Training']['lr_path']
lr_test_path = config['Training']['lr_test_data']
hr_test_path = config['Training']['hr_test_data']

In [7]:
import data_preprocessor as dp
import numpy as np
import skimage as ski
from sklearn.model_selection import train_test_split

img = ski.io.imread(raw_path)

left_over = np.array(img.shape) % im_size
if(left_over[0] != 0):
    img = img[0:-left_over[0],:,:]
if(left_over[1] != 0):
    img = img[:,0:-left_over[1],:]
if(left_over[2] != 0):
    img = img[:,:,0:-left_over[2]]

data = dp.cubify(img, (im_size,im_size,im_size))
train_set, test_set = train_test_split(data,test_size = 0.1)

In [8]:
def salt_data(data):
    down_scaled = []
    for img in data:
        x_center = img.shape[0] // 2
        y_center = img.shape[1] // 2
        z_center = img.shape[2] // 2

        fft_img = np.fft.fftn(img) 

        fft_img[x_center-20 : x_center+20, y_center-20 : y_center+20, z_center-20 : z_center+20] = 0
        inv_img = np.fft.ifftn(fft_img)

        down_sp = ski.transform.resize(abs(inv_img),(12,12,12))
        up_sp = ski.transform.resize(down_sp,img.shape)

        down_scaled.append(abs(up_sp))
    return np.array(down_scaled)

lr_train = salt_data(train_set)
lr_test = salt_data(test_set)

In [9]:
import h5py

file_list = [(hr_path,train_set),(lr_path,lr_train),(lr_test_path,lr_test),(hr_test_path,test_set)]

for name,val in file_list:
    hf = h5py.File(name, "w")
    data_set = hf.create_dataset("data", data=val.astype(np.float64))
    hf.close()

In [10]:
img_hr = h5py.File(hr_path, 'r')
img_lr = h5py.File(lr_path, 'r')

img_hr_data = img_hr['data']
img_lr_data = img_lr['data']

In [11]:
from ipywidgets import interactive

def f(img):
    def func(i,z):
        fig, ax = plt.subplots()
        ax.imshow(img[i,:,:,z],cmap = 'gray')
        plt.show()
    return func

disp_img = f(img_hr_data)

interactive_plot = interactive(disp_img, i=(0, img_hr_data.shape[0]), z=(0,img_hr_data.shape[3]))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(IntSlider(value=12, description='i', max=24), IntSlider(value=64, description='z', max=1…

In [12]:
disp_img_test = f(img_lr_data)

interactive_plot = interactive(disp_img_test, i=(0, img_lr_data.shape[0]), z=(0,img_lr_data.shape[3]))
output = interactive_plot.children[-1]
output.layout.height = '350px'
interactive_plot

interactive(children=(IntSlider(value=12, description='i', max=24), IntSlider(value=64, description='z', max=1…